In [ ]:
# You need to load your OPENAI_API_KEY into your environment
with open('../../my_key.txt') as f:
    key_list = f.readlines()

key_list = [key.strip() for key in key_list]

keys = {}
for key in key_list:
    key = key.split(':')
    if (key != ['']):
        keys[key[0]] = key[1]

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = keys['OPENAI_API_KEY']
os.environ["GOOGLE_CSE_ID"] = keys['GOOGLE_CSE_ID']
os.environ["GOOGLE_API_KEY"] = keys['GOOGLE_API_KEY']

# Custom LLM Agent

In [ ]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, GoogleSearchAPIWrapper, LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
import re

## Tools

In [ ]:
Tools = []

In [ ]:
# Google Search Tool: Useful when you need to answer question about facts
search_engine = GoogleSearchAPIWrapper()
search_tool = Tool(
        name = "Google Search",
        description = "Useful when you need to answer question about facts",
        func = search_engine.run
)

Tools.append(search_tool)

In [ ]:
# Ask User Tool: Useful when you want to ask user for his purpose.
def ask_usr(question: str = None) -> str:
    """Get the user input"""
    user_reply = input(f"{question}: ")
    return user_reply

ask_usr_tool = Tool(
    name = "Ask User",
    description = "Useful when you want to ask user for his purpose.",
    func = ask_usr
)

Tools.append(ask_usr_tool)

## Custom Prompt

In [ ]:
# key of the agent, define what he should do
template = """Your name is iArt.ai Chatbot. You are going to help user design a PPT. You at least need to know what are user's purpose and topic, and your object is to generate an outline with slides' content in detail. 
The user might be unclear about what information should be provided to you, so ask him if you want.
You have access to the following tools:

{tools}

Use the following format:

Initial Idea: The initial idea of user
Thought: you should always think about to design this PPT, what should you know
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: The information is enough for me to design PPT
Final Design: the good-enough PPT outline with slides' content related to the user's purpose and topic

Begin! Remember you are going to generate an outline with slides' content in detail.

Initial Idea:{input}
{agent_scratchpad}
"""

In [ ]:
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)
    
# This omits the 'agent_scratchpad', 'tools', and 'tool_name' variables 
# because those are generated dynamically

# This also includes 'intermediate steps'

In [ ]:
prompt = CustomPromptTemplate(
    template = template,
    tools = Tools,
    input_variables = ["input", "intermediate_steps"]
)

## Output Parser

In [ ]:
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Design:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Design:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [ ]:
output_parser = CustomOutputParser()

## Other Config

In [ ]:
# LLM
openai_llm = OpenAI(temperature = 0)

In [ ]:
# stop sequence
stop_sequence = "\nObservation:"

## Test

In [ ]:
llm_chain = LLMChain(
    llm = openai_llm,
    prompt = prompt
)

In [ ]:
tool_names = [tool.name for tool in Tools]
agent = LLMSingleActionAgent(
    llm_chain = llm_chain,
    output_parser = output_parser,
    stop = [stop_sequence],
    allowed_tools = tool_names
)

In [ ]:
print(tool_names)

In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent = agent,
    tools = Tools,
    verbose = True
)

In [ ]:
result = agent_executor.run("I don't know where to start")

In [ ]:
print(result)